In [1]:
import phyloGraph as ph

from plotly.offline import download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)

#vim ~/.Plotly/.Credentials

## Load Data

In [2]:
pgd = ph.phyloData()

naw


Pick one of the methods below for getting data

### 1) fetch from Tree of Life

In [15]:
pgd.search_name("prima")

,name,id
0,Lamprima,104772
1,Baeotis prima,104881
2,Cryptocercus primarius,8570
3,Procambarus (Austrocambarus) primaevus,7387
4,'Nine-primaried oscines',67287
5,Primates,15963
6,Diopsosoma prima,118152
7,Parameranoplus primaevus,22752
8,Aiptasioides prima,17991
9,Cerapterites primaevus,135132


In [16]:
pgd.fetch_tol_data(15963)

fetching http://tolweb.org/onlinecontributors/app?service=external&page=xml/TreeStructureService&node_id=15963 ..
Fetched 2472 lines. Parsing and assigning...
All done. 166 organisms fetched.


### 2) load from raw file

In [5]:
#raw_file = "data/primate_raw.xml"
#pgd.load_raw_file(raw_file)

In [6]:
#pgd.write_prep_data("data/primate_df.csv", "data/primate_links.txt")

### 3) load from prepared files

In [7]:
#pgd.load_prep_data("data/primate_df.csv", "data/primate_links.txt")

### return data for plotting

In [17]:
df, links_list = pgd.return_data()

## Plot Data

In [18]:
pgp = ph.phyloGraph(df, links_list)

In [19]:
pgp.search_name('rrh')

             name     id
1     Platyrrhini  16292
18     Catarrhini  16293
83  Strepsirrhini  16295


In [20]:
pgp.create_plot_data(16293, max_depth=30, max_nodes=50000)

/Users/Seth/Documents/tolGraph/phyloGraph.py:337: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Loaded plot data. Highlighting 16293


In [21]:
pgp.render_plot()
#pgp.render_plot(publish=True)